## Import Libraries
Import the necessary Python libraries

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import ast

In [18]:
%matplotlib inline
sns.set_style('whitegrid')

## Load Data
Load credits csv file and perform two tasks while loading:

Convert all columns with json data as json type

In [19]:
def load_credits_data(file_path):
    df = pd.read_csv(file_path, dtype='unicode')
    # all json columns`
    json_columns = ['cast', 'crew']
    for column in json_columns:
        # use ast because json data has single quotes in the csv, which is invalid for a json object; it should be " normally
        df[column] = df[column].apply(lambda x: np.nan if pd.isnull(x) else ast.literal_eval(x))
    return df

Load the credits data csv file

In [20]:
credits = load_credits_data(r"D:\uChicago\Classes\Q2\Data Mining\project\the-movies-dataset\credits.csv")

Let's see the basic summay of the available datset

In [21]:
credits.shape

(45476, 3)

In [22]:
credits.columns

Index(['cast', 'crew', 'id'], dtype='object')

In [23]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
cast    45476 non-null object
crew    45476 non-null object
id      45476 non-null object
dtypes: object(3)
memory usage: 1.0+ MB


Check the count of empty cells for each column

In [24]:
credits.isnull().sum()

cast    0
crew    0
id      0
dtype: int64

Create an empty DateFrame credits_flattened to store the de-serialized output. Flatten the actors, directors, producers, casting crew's name and gender information

In [25]:
credits_flattened = pd.DataFrame(None,None,columns=['movie_id','actor_1_gender','actor_2_gender','actor_3_gender',\
                        'actor_4_gender','actor_5_gender','actor_1_name','actor_2_name','actor_3_name','actor_4_name',\
                        'actor_5_name','director_gender','director_name','producer_gender','producer_name','casting_gender',\
                                'casting_name'])

for i,row in credits.iterrows():
    # dummy row
    newrow = {'movie_id':np.nan,'actor_1_gender':np.nan,'actor_2_gender':np.nan,'actor_3_gender':np.nan,'actor_4_gender':np.nan,\
              'actor_5_gender':np.nan,'actor_1_name':np.nan,'actor_2_name':np.nan,'actor_3_name':np.nan,'actor_4_name':np.nan,\
            'actor_5_name':np.nan,'director_gender':np.nan,'director_name':np.nan,'producer_gender':np.nan,\
            'producer_name':np.nan,'casting_gender':np.nan,'casting_name':np.nan}
    
    # fill movie id
    newrow['movie_id'] = int(row['id'])
    
    # fill cast
    count=1
    for item in row['cast']:
        if count==6:
            break
        if 'gender' in item and 'name' in item:
            newrow['actor_'+str(count)+'_gender'] = item['gender']
            newrow['actor_'+str(count)+'_name'] = item['name']
            count += 1

    # fill crew
    director=0
    producer=0
    casting=0
    for item in row['crew']:
        if director and producer and casting:
            break
        if 'job' in item and item['job'] in ['Director','Producer','Casting'] and 'gender' in item and 'name' in item:
            if item['job'] == 'Director' and not director:
                newrow['director_gender'] = item['gender']
                newrow['director_name'] = item['name']
                director =  1
            elif item['job'] == 'Producer' and not producer:
                newrow['producer_gender'] = item['gender']
                newrow['producer_name'] = item['name']
                producer =  1
            elif item['job'] == 'Casting' and not casting:
                newrow['casting_gender'] = item['gender']
                newrow['casting_name'] = item['name']
                casting =  1

    credits_flattened = credits_flattened.append(newrow,ignore_index=True)

Let's verify the structure of the flattened credits data

In [26]:
credits_flattened.shape

(45476, 17)

In [27]:
credits_flattened.head()

,movie_id,actor_1_gender,actor_2_gender,actor_3_gender,actor_4_gender,actor_5_gender,actor_1_name,actor_2_name,actor_3_name,actor_4_name,actor_5_name,director_gender,director_name,producer_gender,producer_name,casting_gender,casting_name
0,862,2,2,2,2,2,Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wallace Shawn,2,John Lasseter,1,Bonnie Arnold,NaN,NaN
1,8844,2,2,1,0,1,Robin Williams,Jonathan Hyde,Kirsten Dunst,Bradley Pierce,Bonnie Hunt,2,Joe Johnston,2,Scott Kroopf,0,Nancy Foy
2,15602,2,2,1,1,1,Walter Matthau,Jack Lemmon,Ann-Margret,Sophia Loren,Daryl Hannah,2,Howard Deutch,NaN,NaN,NaN,NaN
3,31357,1,1,1,1,2,Whitney Houston,Angela Bassett,Loretta Devine,Lela Rochon,Gregory Hines,2,Forest Whitaker,0,Ronald Bass,NaN,NaN
4,11862,2,1,2,1,2,Steve Martin,Diane Keaton,Martin Short,Kimberly Williams-Paisley,George Newbern,2,Charles Shyer,1,Nancy Meyers,NaN,NaN


Save the flattened credits data to a csv file

In [29]:
credits_flattened.to_csv('flattened.csv',index=False,encoding='utf-8')

In [30]:
credits_flattened.head()

,movie_id,actor_1_gender,actor_2_gender,actor_3_gender,actor_4_gender,actor_5_gender,actor_1_name,actor_2_name,actor_3_name,actor_4_name,actor_5_name,director_gender,director_name,producer_gender,producer_name,casting_gender,casting_name
0,862,2,2,2,2,2,Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wallace Shawn,2,John Lasseter,1,Bonnie Arnold,NaN,NaN
1,8844,2,2,1,0,1,Robin Williams,Jonathan Hyde,Kirsten Dunst,Bradley Pierce,Bonnie Hunt,2,Joe Johnston,2,Scott Kroopf,0,Nancy Foy
2,15602,2,2,1,1,1,Walter Matthau,Jack Lemmon,Ann-Margret,Sophia Loren,Daryl Hannah,2,Howard Deutch,NaN,NaN,NaN,NaN
3,31357,1,1,1,1,2,Whitney Houston,Angela Bassett,Loretta Devine,Lela Rochon,Gregory Hines,2,Forest Whitaker,0,Ronald Bass,NaN,NaN
4,11862,2,1,2,1,2,Steve Martin,Diane Keaton,Martin Short,Kimberly Williams-Paisley,George Newbern,2,Charles Shyer,1,Nancy Meyers,NaN,NaN


Function to load movies data. This function converts date column to datetime.date data type and also assign json data type to corresponding json columns

In [31]:
def load_movies_metadata(file_path):
    df = pd.read_csv(file_path, dtype='unicode')
    # covert each item of release_date to datetime.date type entity
    df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce').apply(lambda x: x.date())
    # all json columns`
    json_columns = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']
    for column in json_columns:
        # use ast because json data has single quotes in the csv, which is invalid for a json object; it should be " normally
        df[column] = df[column].apply(lambda x: np.nan if pd.isnull(x) else ast.literal_eval(x))
    return df

Load movies metadata using function defined above

In [32]:
movies = load_movies_metadata(r"D:\uChicago\Classes\Q2\Data Mining\project\the-movies-dataset\movies_metadata.csv")

Verify movies metadata information

In [33]:
movies.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92


In [34]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

Create a new dataframe to store only the desired columns from the movies metadata loaded from csv

In [285]:
movies_required = movies[['budget', 'genres', 'id', 'imdb_id', 'original_language', 'title', 'popularity', 'release_date', 'revenue','vote_average','vote_count']]

In [286]:
movies_required.head()

,budget,genres,id,imdb_id,original_language,title,popularity,release_date,revenue,vote_average,vote_count
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,en,Toy Story,21.946943,1995-10-30,373554033,7.7,5415
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,en,Jumanji,17.015539,1995-12-15,262797249,6.9,2413
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,en,Grumpier Old Men,11.7129,1995-12-22,0,6.5,92
3,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,en,Waiting to Exhale,3.859495,1995-12-22,81452156,6.1,34
4,0,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,en,Father of the Bride Part II,8.387519,1995-02-10,76578911,5.7,173


In [287]:
movies_required.shape

(45466, 11)

Keep only the first genre information out of all the json values

In [288]:
for i,row in movies_required.iterrows():
    if len(row[1])>0:
        if 'name' in row[1][0]:
            row[1] = row[1][0]['name']

In [289]:
movies_required.head()

,budget,genres,id,imdb_id,original_language,title,popularity,release_date,revenue,vote_average,vote_count
0,30000000,Animation,862,tt0114709,en,Toy Story,21.946943,1995-10-30,373554033,7.7,5415
1,65000000,Adventure,8844,tt0113497,en,Jumanji,17.015539,1995-12-15,262797249,6.9,2413
2,0,Romance,15602,tt0113228,en,Grumpier Old Men,11.7129,1995-12-22,0,6.5,92
3,16000000,Comedy,31357,tt0114885,en,Waiting to Exhale,3.859495,1995-12-22,81452156,6.1,34
4,0,Comedy,11862,tt0113041,en,Father of the Bride Part II,8.387519,1995-02-10,76578911,5.7,173


Rename columns appropriately

In [290]:
movies_required.rename(columns={'genres':'genre_top','id':'movie_id'}, inplace=True)

C:\Users\samsu\Anaconda3\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [291]:
movies_required.head()

,budget,genre_top,movie_id,imdb_id,original_language,title,popularity,release_date,revenue,vote_average,vote_count
0,30000000,Animation,862,tt0114709,en,Toy Story,21.946943,1995-10-30,373554033,7.7,5415
1,65000000,Adventure,8844,tt0113497,en,Jumanji,17.015539,1995-12-15,262797249,6.9,2413
2,0,Romance,15602,tt0113228,en,Grumpier Old Men,11.7129,1995-12-22,0,6.5,92
3,16000000,Comedy,31357,tt0114885,en,Waiting to Exhale,3.859495,1995-12-22,81452156,6.1,34
4,0,Comedy,11862,tt0113041,en,Father of the Bride Part II,8.387519,1995-02-10,76578911,5.7,173


Below columns had to be dropped because the order of values in these rows was incorrect

In [292]:
movies_required.drop(movies_required.index[19730],inplace=True)

C:\Users\samsu\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [293]:
movies_required.drop(movies_required.index[29502],inplace=True)

C:\Users\samsu\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [294]:
movies_required.drop(movies_required.index[35585],inplace=True)

C:\Users\samsu\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Convert movie_id data type to numeric value instead of the default 'string'

In [295]:
movies_required['movie_id'] = pd.to_numeric(movies_required['movie_id'])

C:\Users\samsu\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [296]:
movies_required.head()

,budget,genre_top,movie_id,imdb_id,original_language,title,popularity,release_date,revenue,vote_average,vote_count
0,30000000,Animation,862,tt0114709,en,Toy Story,21.946943,1995-10-30,373554033,7.7,5415
1,65000000,Adventure,8844,tt0113497,en,Jumanji,17.015539,1995-12-15,262797249,6.9,2413
2,0,Romance,15602,tt0113228,en,Grumpier Old Men,11.7129,1995-12-22,0,6.5,92
3,16000000,Comedy,31357,tt0114885,en,Waiting to Exhale,3.859495,1995-12-22,81452156,6.1,34
4,0,Comedy,11862,tt0113041,en,Father of the Bride Part II,8.387519,1995-02-10,76578911,5.7,173


Merge the two dataframes to get the desired result

In [297]:
merged_df = pd.merge(credits_flattened,movies_required,on=['movie_id'])

In [298]:
merged_df.shape

(45538, 27)

In [299]:
merged_df.columns

Index(['movie_id', 'actor_1_gender', 'actor_2_gender', 'actor_3_gender',
       'actor_4_gender', 'actor_5_gender', 'actor_1_name', 'actor_2_name',
       'actor_3_name', 'actor_4_name', 'actor_5_name', 'director_gender',
       'director_name', 'producer_gender', 'producer_name', 'casting_gender',
       'casting_name', 'budget', 'genre_top', 'imdb_id', 'original_language',
       'title', 'popularity', 'release_date', 'revenue', 'vote_average',
       'vote_count'],
      dtype='object')

In [300]:
merged_df.head()

,movie_id,actor_1_gender,actor_2_gender,actor_3_gender,actor_4_gender,actor_5_gender,actor_1_name,actor_2_name,actor_3_name,actor_4_name,...,budget,genre_top,imdb_id,original_language,title,popularity,release_date,revenue,vote_average,vote_count
0,862,2,2,2,2,2,Tom Hanks,Tim Allen,Don Rickles,Jim Varney,...,30000000,Animation,tt0114709,en,Toy Story,21.946943,1995-10-30,373554033,7.7,5415
1,8844,2,2,1,0,1,Robin Williams,Jonathan Hyde,Kirsten Dunst,Bradley Pierce,...,65000000,Adventure,tt0113497,en,Jumanji,17.015539,1995-12-15,262797249,6.9,2413
2,15602,2,2,1,1,1,Walter Matthau,Jack Lemmon,Ann-Margret,Sophia Loren,...,0,Romance,tt0113228,en,Grumpier Old Men,11.7129,1995-12-22,0,6.5,92
3,31357,1,1,1,1,2,Whitney Houston,Angela Bassett,Loretta Devine,Lela Rochon,...,16000000,Comedy,tt0114885,en,Waiting to Exhale,3.859495,1995-12-22,81452156,6.1,34
4,11862,2,1,2,1,2,Steve Martin,Diane Keaton,Martin Short,Kimberly Williams-Paisley,...,0,Comedy,tt0113041,en,Father of the Bride Part II,8.387519,1995-02-10,76578911,5.7,173


Write the merged result to a csv file

In [301]:
merged_df.to_csv('flattened_merged.csv',index=False)

In [48]:
movies.drop(movies.index[19730],inplace=True)

In [50]:
movies.drop(movies.index[29502],inplace=True)

In [52]:
movies.drop(movies.index[35585],inplace=True)

In [45]:
movies.rename(columns={'id':'movie_id'}, inplace=True)

In [53]:
movies['movie_id'] = pd.to_numeric(movies['movie_id'])

In [56]:
merged_all = pd.merge(credits_flattened,movies,on=['movie_id'])

In [57]:
merged_all.shape

(45538, 40)

In [58]:
merged_all.to_csv('flattened_merged_master.csv',index=False,encoding='utf8')

We shall use this csv file for our modelling purposes.